In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from glob import glob
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from keras.models import load_model
import pandas as pd
from tensorflow.python.client import device_lib

In [ ]:
print(device_lib.list_local_devices())

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
imgs = glob("C:/Users/hsj/Desktop/image/*/*.png")

# classification dataset (train and test)

In [9]:
train_img = tf.keras.preprocessing.image_dataset_from_directory(
    directory = "D:/image",
    validation_split=0.2,
    subset = "training",
    seed =1337,
    image_size=(500,500),
    batch_size=16
)
test_img = tf.keras.preprocessing.image_dataset_from_directory(
    directory="D:/image",
    validation_split=0.2,
    subset = "validation",
    seed =1337,
    image_size=(500,500),
    batch_size=16
)


Found 5766 files belonging to 144 classes.
Using 4613 files for training.
Found 5766 files belonging to 144 classes.
Using 1153 files for validation.


In [ ]:
class_names = test_img.class_names
plt.figure(figsize=(10,10))
for images, labels in train_img.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("on")

In [ ]:
class_names

In [ ]:
for image_batch, labels_batch in train_img:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

normalization_layer = tf.keras.layers.Rescaling(1./255)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_img = train_img.cache().prefetch(buffer_size=AUTOTUNE)
test_img = test_img.cache().prefetch(buffer_size= AUTOTUNE)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(500, 500, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(144, activation='softmax')
])

In [ ]:
print('Train:Test = {}:{}'.format(len(train_img), len(test_img)))

In [ ]:
early_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)

In [ ]:
model.summary()

model.compile(loss = 'sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

history = model.fit(train_img, epochs=36,  validation_data = test_img, verbose = 1, validation_steps=30, callbacks=[early_callback])

model.save("rps4.h5")

In [ ]:

img = keras.preprocessing.image.load_img("image/198701126/198701126_0_26.png", target_size=(500,180))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array).flatten()
score = predictions[0]

print("This image is {:.2f} percent 198701126" .format(100*score))


In [11]:
model.evaluate(test_img)

73/73 [==============================] - 29s 390ms/step - loss: 0.1981 - accuracy: 0.9454


[0.19812703132629395, 0.9453599452972412]

In [12]:
model.evaluate(train_img)

289/289 [==============================] - 114s 390ms/step - loss: 0.0640 - accuracy: 0.9820


[0.06396707147359848, 0.9820073843002319]

In [ ]:
reconstructed_model = load_model("rps3.h5")

In [ ]:
reconstructed_model.evaluate(test_img)

In [14]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

NameError: name 'history' is not defined

In [10]:
model = load_model("rps3.h5")

In [15]:
#predictions = model.predict(test_img)
img = keras.preprocessing.image.load_img("image/198701126/198701126_0_21.png", target_size=(180,180))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis
predictions = model.predict(img_array)

plt.imshow(img)
class_names[np.argmax(predictions)]

FileNotFoundError: [Errno 2] No such file or directory: 'image/198701126/198701126_0_21.png'